In [38]:
import requests
import csv
from google.colab import files  # Colab'dan indirme işlemi için

In [39]:
# TMDb API ayarları
API_KEY = "bb8429d2796b5eb1b2cd6a9c59297e67"  # Buraya kendi API anahtarınızı girin
BASE_URL = "https://api.themoviedb.org/3"
LANGUAGE = "tr-TR"  # Türkçe sonuçlar için

In [40]:
# CSV dosyasına yazma fonksiyonu
def write_to_csv(file_name, data):
    with open(file_name, mode="w", newline="", encoding="utf-8") as file:
        writer = csv.writer(file)
        # Başlıklar
        writer.writerow(["Film Adı", "Türler", "Özet"])
        # Veriler
        for row in data:
            writer.writerow(row)

In [41]:
# Filmleri al ve veri seti oluştur
def fetch_movies(page_limit=10):
    all_movies = []

    for page in range(1, page_limit + 1):
        url = f"{BASE_URL}/movie/popular"
        params = {
            "api_key": API_KEY,
            "language": LANGUAGE,
            "page": page
        }

        response = requests.get(url, params=params)
        if response.status_code == 200:
            data = response.json()
            for movie in data.get("results", []):
                title = movie.get("title")
                overview = movie.get("overview", "Özet bulunamadı")
                genre_ids = movie.get("genre_ids", [])

                # Tür bilgilerini al
                genres = fetch_genres(genre_ids)

                # Veriyi listeye ekle
                all_movies.append([title, ", ".join(genres), overview])
        else:
            print(f"API hatası: {response.status_code}")
            break

    return all_movies

In [42]:
# Tür ID'lerini al ve eşleştir
def fetch_genres(genre_ids):
    url = f"{BASE_URL}/genre/movie/list"
    params = {
        "api_key": API_KEY,
        "language": LANGUAGE
    }

    response = requests.get(url, params=params)
    if response.status_code == 200:
        genres_data = response.json().get("genres", [])
        genres_dict = {genre["id"]: genre["name"] for genre in genres_data}
        return [genres_dict.get(genre_id, "Bilinmeyen Tür") for genre_id in genre_ids]
    else:
        print(f"Tür bilgisi alınamadı: {response.status_code}")
        return []


In [44]:
# Veri setini oluştur ve kaydet
def main():
    print("Filmler alınıyor...")
    movies = fetch_movies(page_limit=500)  # İlk 500 sayfa için (her sayfa 20 film içerir)

    # İlk filmi kontrol edin
    if movies:
        print("Örnek bir film verisi:", movies[0])

    print(f"{len(movies)} film alındı.")

    file_name = "turkce_filmler.csv"
    write_to_csv(file_name, movies)
    print(f"Veri seti {file_name} dosyasına kaydedildi.")

    # Colab üzerinden indirme
    files.download(file_name)
    print(f"{file_name} dosyası indirilmeye hazır.")

if __name__ == "__main__":
    main()

Filmler alınıyor...
Örnek bir film verisi: ['Gladyatör II', 'Aksiyon, Macera, Dram', 'Gladyatör Maximus Decimus Meridius’un ölümünden 15 yıl sonrasıdır. Kana susamış kalabalıklar yarı deli ikiz imparatorları zalimliğin yeni zirvelerine iterken Kolezyum’daki savaşlar çok daha ölümcül ve görkemli hâle gelmiştir. Roma İmparatorluğu sınırlarını genişleterek, yoluna çıkan her kültürü yağmalayarak ve hayatta kalanları ringde yaşamları için savaşmaya zorlayarak yeryüzündeki yürüyüşünü sürdürmektedir.']
10000 film alındı.
Veri seti turkce_filmler.csv dosyasına kaydedildi.


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

turkce_filmler.csv dosyası indirilmeye hazır.
